In [1]:
from ucimlrepo import fetch_ucirepo 
import numpy as np 

In [2]:
auction_verification = fetch_ucirepo(id=713) 

In [3]:
train = auction_verification.data.features 
result = auction_verification.data.targets 

In [4]:
train.head()

,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner
0,0,0,2,1,59,1,0
1,0,0,2,1,59,2,0
2,0,0,2,1,59,4,0
3,0,0,2,1,59,6,0
4,0,0,2,1,60,1,0


In [5]:
result.head()

,verification.result,verification.time
0,False,163.316667
1,False,200.860000
2,False,154.888889
3,False,108.640000
4,True,85.466667


In [6]:
result['verification.result'].value_counts()

verification.result
False    1781
True      262
Name: count, dtype: int64

In [7]:
result['verification.result'] = result['verification.result'].astype(int)
result.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2864\2159149282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['verification.result'] = result['verification.result'].astype(int)


,verification.result,verification.time
0,0,163.316667
1,0,200.860000
2,0,154.888889
3,0,108.640000
4,1,85.466667


In [8]:
train['verification.time'] = result['verification.time']

C:\Users\User\AppData\Local\Temp\ipykernel_2864\289437493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['verification.time'] = result['verification.time']


In [9]:
result.rename(columns={'verification.result': 'target'}, inplace = True)

C:\Users\User\AppData\Local\Temp\ipykernel_2864\1257734431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.rename(columns={'verification.result': 'target'}, inplace = True)


In [10]:
result.drop('verification.time', axis=1, inplace=True)
result.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2864\261690860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop('verification.time', axis=1, inplace=True)


,target
0,0
1,0
2,0
3,0
4,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train, result['target'], test_size=0.3, random_state=42)

In [13]:
base_learners = [
    ('decision_tree', DecisionTreeClassifier(max_depth=1)),
    ('lr', LogisticRegression())
]

In [14]:
meta_learner = SVC(probability=True, random_state=42)

In [15]:
stack_clf = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)

In [16]:
stack_clf.fit(X_train, y_train)

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

StackingClassifier(cv=5,
                   estimators=[('decision_tree',
                                DecisionTreeClassifier(max_depth=1)),
                               ('lr', LogisticRegression())],
                   final_estimator=SVC(probability=True, random_state=42))

In [17]:
predictions = stack_clf.predict(X_test)

In [18]:
print("Stacking Model Accuracy:", accuracy_score(y_test, predictions))

Stacking Model Accuracy: 0.9233278955954323


In [19]:
from sklearn.ensemble import AdaBoostClassifier

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train, result['target'], test_size=0.3, random_state=42)

In [21]:
shallow_decision_tree = DecisionTreeClassifier(max_depth=1)

In [22]:
ada_clf = AdaBoostClassifier(estimator=shallow_decision_tree, n_estimators=100, random_state=42, algorithm="SAMME")

In [23]:
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=100, random_state=42)

In [24]:
predictions = ada_clf.predict(X_test)

In [25]:
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.9869494290375204


In [26]:
from sklearn.ensemble import BaggingClassifier

In [27]:
bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=2, max_features=1),
    n_estimators=10,
    random_state=42
)

In [28]:
bagging_model.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=2, max_features=1),
                  random_state=42)

In [29]:
predictions = bagging_model.predict(X_test)

In [30]:
accuracy = accuracy_score(y_test, predictions)

In [31]:
print(f'Accuracy of scikit-learn BaggingClassifier: {accuracy:.2f}')

Accuracy of scikit-learn BaggingClassifier: 0.87
